In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime
from skimage import exposure

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [5]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= 255
x_test /= 255

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [10]:
validation_split = int((1-VAL_SPLIT)*len(x_train))
x_train, x_val = x_train[:validation_split], x_train[validation_split:]
y_train, y_val = y_train[:validation_split], y_train[validation_split:]

In [11]:
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

In [12]:
datagen = ImageDataGenerator(preprocessing_function=contrast_stretching)

# Build model

In [13]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [14]:
base_model = VGG19(include_top=False, input_tensor=input_tensor)

In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [17]:
vgg19 = Model(inputs=base_model.input, outputs=predictions)

In [18]:
vgg19.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [19]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [21]:
vgg19.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, 
                    validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/40
79/78 [==============================] - 59s 745ms/step - loss: 2.0591 - acc: 0.2476 - val_loss: 1.9725 - val_acc: 0.2938
Epoch 2/40
79/78 [==============================] - 49s 626ms/step - loss: 1.5635 - acc: 0.4470 - val_loss: 1.6094 - val_acc: 0.4299
Epoch 3/40
79/78 [==============================] - 51s 643ms/step - loss: 1.3105 - acc: 0.5459 - val_loss: 1.2013 - val_acc: 0.6102
Epoch 4/40
79/78 [==============================] - 51s 640ms/step - loss: 1.1664 - acc: 0.5990 - val_loss: 1.0533 - val_acc: 0.6455
Epoch 5/40
79/78 [==============================] - 51s 644ms/step - loss: 1.0740 - acc: 0.6318 - val_loss: 1.0856 - val_acc: 0.6262
Epoch 6/40
79/78 [==============================] - 51s 643ms/step - loss: 0.9825 - acc: 0.6648 - val_loss: 1.0472 - val_acc: 0.6512
Epoch 7/40
79/78 [==============================] - 51s 641ms/step - loss: 0.9531 - acc: 0.6741 - val_loss: 1.0299 - val_acc: 0.6584
Epoch 8/40
79/78 [==============================] - 51s 642ms/step - 

In [22]:
vgg19.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 7s 709us/step


[1.1285193413734436, 0.6786]